In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import easyocr

AttributeError: partially initialized module 'cv2' has no attribute 'gapi_wip_gst_GStreamerPipeline' (most likely due to a circular import)